# day 11

https://adventofcode.com/2018/day/11

In [ ]:
import os

import eri.logging as logging

In [ ]:
LOGGER = logging.getLogger('day11')
logging.configure()

## part 1

### problem statement:

> You watch the Elves and their sleigh fade into the distance as they head toward the North Pole.
> 
> Actually, you're the one fading. The falling sensation returns.
> 
> The low fuel warning light is illuminated on your wrist-mounted device. Tapping it once causes it to project a hologram of the situation: a 300x300 grid of fuel cells and their current power levels, some negative. You're not sure what negative power means in the context of time travel, but it can't be good.
> 
> Each fuel cell has a coordinate ranging from 1 to 300 in both the X (horizontal) and Y (vertical) direction. In X,Y notation, the top-left cell is 1,1, and the top-right cell is 300,1.
> 
> The interface lets you select any 3x3 square of fuel cells. To increase your chances of getting to your destination, you decide to choose the 3x3 square with the largest total power.
> 
> The power level in a given fuel cell can be found through the following process:
> 
> + Find the fuel cell's rack ID, which is its X coordinate plus 10.
> + Begin with a power level of the rack ID times the Y coordinate.
> + Increase the power level by the value of the grid serial number (your puzzle input).
> + Set the power level to itself multiplied by the rack ID.
> + Keep only the hundreds digit of the power level (so 12345 becomes 3; numbers with no hundreds digit become 0).
> + Subtract 5 from the power level.
> 
> For example, to find the power level of the fuel cell at 3,5 in a grid with serial number 8:
> 
> + The rack ID is 3 + 10 = 13.
> + The power level starts at 13 * 5 = 65.
> + Adding the serial number produces 65 + 8 = 73.
> + Multiplying by the rack ID produces 73 * 13 = 949.
> + The hundreds digit of 949 is 9.
> + Subtracting 5 produces 9 - 5 = 4.
> 
> So, the power level of this fuel cell is 4.
> 
> Here are some more example power levels:
> 
> + Fuel cell at  122,79, grid serial number 57: power level -5.
> + Fuel cell at 217,196, grid serial number 39: power level  0.
> + Fuel cell at 101,153, grid serial number 71: power level  4.
> 
> Your goal is to find the 3x3 square which has the largest total power. The square must be entirely within the 300x300 grid. Identify this square using the X,Y coordinate of its top-left fuel cell. For example:
> 
> For grid serial number 18, the largest total 3x3 square has a top-left corner of 33,45 (with a total power of 29); these fuel cells appear in the middle of this 5x5 region:
> 
>     -2  -4   4   4   4
>     -4   4   4   4  -5
>      4   3   3   4  -4
>      1   1   2   4  -3
>     -1   0   2  -5  -2
> 
> For grid serial number 42, the largest 3x3 square's top-left is 21,61 (with a total power of 30); they are in the middle of this region:
> 
>     -3   4   2   2   2
>     -4   4   3   3   4
>     -5   3   3   4  -4
>      4   3   3   4  -3
>      3   3   3  -5  -1
> 
> What is the X,Y coordinate of the top-left fuel cell of the 3x3 square with the largest total power?
> 
> Your puzzle input is 9435.

#### loading data

In [ ]:
def load_data():
    return 9435

#### function def

In [ ]:
def power(x, y, serial_number):
    rack_id = x + 10
    p = rack_id * y
    p += serial_number
    p *= rack_id
    p //= 100
    p %= 10
    p -= 5
    return p

In [ ]:
# tests
assert power(3, 5, 8) == 4
assert power(122, 79, 57) == -5
assert power(217, 196, 39) == 0
assert power(101, 153, 71) == 4

In [ ]:
logging.getLogger('matplotlib').setLevel(logging.ERROR)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def grid(x_max=300, y_max=300, serial_number=0):
    df = pd.DataFrame([
        {'x': x, 'y': y, 'power': power(x, y, serial_number)}
        for x in range(1, x_max + 1)
        for y in range(1, x_max + 1)
    ])
    
    return df.pivot(index='y', columns='x', values='power')

##### example 1

In [ ]:
g = grid(serial_number=18)

In [ ]:
g.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.matshow(g.values);

In [ ]:
g.loc[44: 48, 32: 36]

getting the 3x3 sums...

In [ ]:
conv_mat = np.ones((3, 3))

In [ ]:
from scipy import signal

In [ ]:
z = g.loc[44: 48, 32: 36].values.copy()
z

In [ ]:
signal.convolve2d(g, conv_mat, mode='valid').shape

In [ ]:
def total_power(df, conv_mat):
    return pd.DataFrame(
        signal.convolve2d(df, conv_mat, mode='valid'),
        index=df.index[:-conv_mat.shape[0] + 1],
        columns=df.columns[:-conv_mat.shape[1] + 1],
    )

In [ ]:
tp = total_power(g, conv_mat)

In [ ]:
assert tp.loc[45, 33] == 29

In [ ]:
tp.max().idxmax()

In [ ]:
def max_power(tp):
    """return the x, y values of max tp"""
    xmax = tp.max().idxmax()
    ymax = tp.max(axis=1).idxmax()
    return xmax, ymax

In [ ]:
max_power(tp)

In [ ]:
assert max_power(tp) == (33, 45)

##### example 2

In [ ]:
g = grid(serial_number=42)
tp = total_power(g, conv_mat)
xmax, ymax = max_power(tp)

assert xmax == 21
assert ymax == 61
assert tp.loc[ymax, xmax] == 30

#### answer

In [ ]:
g = grid(serial_number=load_data())
tp = total_power(g, conv_mat)
xmax, ymax = max_power(tp)
xmax, ymax

## part 2

### problem statement:

> You discover a dial on the side of the device; it seems to let you select a square of any size, not just 3x3. Sizes from 1x1 to 300x300 are supported.
> 
> Realizing this, you now must find the square of any size with the largest total power. Identify this square by including its size as a third parameter after the top-left coordinate: a 9x9 square with a top-left corner of 3,5 is identified as 3,5,9.
> 
> For example:
> 
> + For grid serial number 18, the largest total square (with a total power of 113) is 16x16 and has a top-left corner of 90,269, so its identifier is 90,269,16.
> + For grid serial number 42, the largest total square (with a total power of 119) is 12x12 and has a top-left corner of 232,251, so its identifier is 232,251,12.
> 
> What is the X,Y,size identifier of the square with the largest total power?

#### function def

In [ ]:
import tqdm

def q_2(data):
    g = grid(serial_number=data)
    history = []
    for i in tqdm.tnrange(2, 300):
        conv_mat = np.ones((i, i))
        tp = total_power(g, conv_mat)
        xmax, ymax = max_power(tp)
        tp = tp.loc[ymax, xmax]
        history.append({
            'grid_size': i,
            'total_power': tp,
            'xmax': xmax,
            'ymax': ymax
        })
    history = pd.DataFrame(history)
    return history

##### examples

In [ ]:
df_hist = q_2(18)

In [ ]:
df_hist.loc[df_hist.total_power.idxmax()]

In [ ]:
df_hist = q_2(42)

In [ ]:
df_hist.loc[df_hist.total_power.idxmax()]

#### answer

In [ ]:
df_hist = q_2(load_data())

In [ ]:
df_hist.loc[df_hist.total_power.idxmax()]

In [ ]:
'{2:.0f},{3:.0f},{0:.0f}'.format(*(df_hist.loc[df_hist.total_power.idxmax()]))

fin